In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from tqdm import tqdm
import spacy
from gensim import corpora, models

import sys
sys.path.append('..')
from utils import preprocess, get_windows

In [15]:
MIN_COUNTS = 20
MAX_COUNTS = 1800
# words with count < MIN_COUNTS
# and count > MAX_COUNTS
# will be removed

MIN_LENGTH = 15
# minimum document length 
# (number of words)
# after preprocessing

# half the size of the context around a word
HALF_WINDOW_SIZE = 5
# it must be that 2*HALF_WINDOW_SIZE < MIN_LENGTH

# Load NLP model

In [16]:
nlp = spacy.load('en')

# Load dataset

In [17]:
dataset = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
docs = dataset['data']

In [18]:
# number of documents
len(docs)

18846

In [19]:
# store an index with a document
docs = [(i, doc) for i, doc in enumerate(docs)]

# Preprocess dataset and create windows

In [20]:
encoded_docs, decoder, word_counts = preprocess(
    docs, nlp, MIN_LENGTH, MIN_COUNTS, MAX_COUNTS
)



  0%|                                                                                        | 0/18846 [00:00<?, ?it/s]

  0%|                                                                              | 1/18846 [00:00<2:24:48,  2.17it/s]

  0%|                                                                              | 3/18846 [00:00<1:51:56,  2.81it/s]

  0%|                                                                              | 5/18846 [00:00<1:24:57,  3.70it/s]

  0%|                                                                              | 8/18846 [00:01<1:05:49,  4.77it/s]

  0%|                                                                               | 10/18846 [00:01<52:00,  6.04it/s]

  0%|                                                                               | 13/18846 [00:01<53:11,  5.90it/s]

  0%|                                                                               | 16/18846 [00:01<42:01,  7.47it/s]

  0%|                         

  1%|▋                                                                             | 159/18846 [00:12<19:50, 15.70it/s]

  1%|▋                                                                             | 163/18846 [00:13<16:41, 18.66it/s]

  1%|▋                                                                             | 166/18846 [00:13<16:13, 19.19it/s]

  1%|▋                                                                             | 169/18846 [00:13<15:05, 20.63it/s]

  1%|▋                                                                             | 172/18846 [00:13<16:34, 18.79it/s]

  1%|▋                                                                             | 176/18846 [00:13<15:30, 20.05it/s]

  1%|▋                                                                             | 179/18846 [00:13<21:02, 14.79it/s]

  1%|▊                                                                             | 183/18846 [00:14<22:02, 14.11it/s]

  1%|▊                          

  2%|█▍                                                                            | 348/18846 [00:25<22:08, 13.92it/s]

  2%|█▍                                                                            | 352/18846 [00:26<19:54, 15.48it/s]

  2%|█▍                                                                            | 355/18846 [00:26<20:45, 14.85it/s]

  2%|█▍                                                                            | 357/18846 [00:26<20:15, 15.21it/s]

  2%|█▍                                                                            | 360/18846 [00:26<17:50, 17.27it/s]

  2%|█▍                                                                            | 362/18846 [00:26<20:09, 15.28it/s]

  2%|█▌                                                                            | 366/18846 [00:26<17:33, 17.55it/s]

  2%|█▌                                                                            | 368/18846 [00:26<18:45, 16.42it/s]

  2%|█▌                         

  3%|██                                                                            | 513/18846 [00:39<23:12, 13.17it/s]

  3%|██▏                                                                           | 515/18846 [00:39<26:33, 11.51it/s]

  3%|██▏                                                                           | 518/18846 [00:39<22:11, 13.77it/s]

  3%|██▏                                                                           | 521/18846 [00:39<20:52, 14.63it/s]

  3%|██▏                                                                           | 523/18846 [00:40<26:03, 11.72it/s]

  3%|██▏                                                                           | 526/18846 [00:40<22:45, 13.42it/s]

  3%|██▏                                                                           | 529/18846 [00:40<19:15, 15.85it/s]

  3%|██▏                                                                           | 531/18846 [00:40<18:17, 16.69it/s]

  3%|██▏                        

  4%|██▊                                                                           | 688/18846 [00:56<50:40,  5.97it/s]

  4%|██▊                                                                           | 693/18846 [00:56<38:26,  7.87it/s]

  4%|██▉                                                                           | 696/18846 [00:56<36:31,  8.28it/s]

  4%|██▉                                                                           | 699/18846 [00:56<28:55, 10.46it/s]

  4%|██▉                                                                           | 702/18846 [00:57<28:02, 10.78it/s]

  4%|██▉                                                                           | 704/18846 [00:57<24:26, 12.37it/s]

  4%|██▉                                                                           | 707/18846 [00:57<21:20, 14.17it/s]

  4%|██▉                                                                           | 710/18846 [00:57<19:00, 15.90it/s]

  4%|██▉                        

...

 97%|█████████████████████████████████████████████████████████████████████████▌  | 18257/18846 [23:02<00:52, 11.16it/s]

 97%|█████████████████████████████████████████████████████████████████████████▋  | 18260/18846 [23:02<00:43, 13.47it/s]

 97%|█████████████████████████████████████████████████████████████████████████▋  | 18262/18846 [23:02<01:10,  8.29it/s]

 97%|█████████████████████████████████████████████████████████████████████████▋  | 18265/18846 [23:02<00:56, 10.26it/s]

 97%|█████████████████████████████████████████████████████████████████████████▋  | 18269/18846 [23:02<00:44, 13.04it/s]

 97%|█████████████████████████████████████████████████████████████████████████▋  | 18273/18846 [23:02<00:36, 15.74it/s]

 97%|█████████████████████████████████████████████████████████████████████████▋  | 18276/18846 [23:03<00:32, 17.53it/s]

 97%|█████████████████████████████████████████████████████████████████████████▋  | 18279/18846 [23:03<00:30, 18.71it/s]

 97%|███████████████████████████

 98%|██████████████████████████████████████████████████████████████████████████▍ | 18468/18846 [23:19<01:00,  6.23it/s]

 98%|██████████████████████████████████████████████████████████████████████████▍ | 18470/18846 [23:19<00:52,  7.11it/s]

 98%|██████████████████████████████████████████████████████████████████████████▍ | 18472/18846 [23:19<00:42,  8.74it/s]

 98%|██████████████████████████████████████████████████████████████████████████▍ | 18474/18846 [23:19<00:37, 10.02it/s]

 98%|██████████████████████████████████████████████████████████████████████████▌ | 18478/18846 [23:19<00:29, 12.45it/s]

 98%|██████████████████████████████████████████████████████████████████████████▌ | 18481/18846 [23:20<00:30, 12.10it/s]

 98%|██████████████████████████████████████████████████████████████████████████▌ | 18483/18846 [23:20<00:26, 13.56it/s]

 98%|██████████████████████████████████████████████████████████████████████████▌ | 18485/18846 [23:20<00:28, 12.53it/s]

 98%|███████████████████████████

 99%|███████████████████████████████████████████████████████████████████████████▏| 18650/18846 [23:30<00:11, 16.92it/s]

 99%|███████████████████████████████████████████████████████████████████████████▏| 18653/18846 [23:30<00:10, 17.99it/s]

 99%|███████████████████████████████████████████████████████████████████████████▏| 18656/18846 [23:31<00:10, 17.86it/s]

 99%|███████████████████████████████████████████████████████████████████████████▏| 18659/18846 [23:31<00:10, 18.35it/s]

 99%|███████████████████████████████████████████████████████████████████████████▎| 18661/18846 [23:31<00:11, 16.53it/s]

 99%|███████████████████████████████████████████████████████████████████████████▎| 18665/18846 [23:31<00:09, 19.64it/s]

 99%|███████████████████████████████████████████████████████████████████████████▎| 18668/18846 [23:31<00:10, 17.14it/s]

 99%|███████████████████████████████████████████████████████████████████████████▎| 18671/18846 [23:31<00:09, 18.93it/s]

 99%|███████████████████████████

100%|███████████████████████████████████████████████████████████████████████████▉| 18841/18846 [23:43<00:00, 11.01it/s]

100%|███████████████████████████████████████████████████████████████████████████▉| 18843/18846 [23:43<00:00, 11.60it/s]

100%|████████████████████████████████████████████████████████████████████████████| 18846/18846 [23:44<00:00, 13.23it/s]


number of removed short documents: 3975
total number of tokens: 1442896
number of tokens to be removed: 412625
number of additionally removed short documents: 2172
total number of tokens: 1005435

minimum word count number: 13
this number can be less than MIN_COUNTS because of document removal


In [21]:
# new ids will be created for the documents.
# create a way of restoring initial ids:
doc_decoder = {i: doc_id for i, (doc_id, doc) in enumerate(encoded_docs)}

In [22]:
data = []
# new ids are created here
for index, (_, doc) in tqdm(enumerate(encoded_docs)):
    windows = get_windows(doc, HALF_WINDOW_SIZE)
    # index represents id of a document, 
    # windows is a list of (word, window around this word),
    # where word is in the document
    data += [[index, w[0]] + w[1] for w in windows]

data = np.array(data, dtype='int64')



0it [00:00, ?it/s]

723it [00:00, 7229.59it/s]

940it [00:00, 3443.72it/s]

1592it [00:00, 4011.49it/s]

1927it [00:00, 2597.68it/s]

2439it [00:00, 3048.15it/s]

2787it [00:00, 2592.75it/s]

3295it [00:00, 3028.25it/s]

3655it [00:01, 2497.92it/s]

4229it [00:01, 3007.51it/s]

4756it [00:01, 2478.08it/s]

5328it [00:01, 2985.72it/s]

5742it [00:01, 3258.21it/s]

6277it [00:01, 3691.13it/s]

6720it [00:02, 2791.14it/s]

7249it [00:02, 3228.14it/s]

7938it [00:02, 3840.43it/s]

8490it [00:02, 2951.58it/s]

9128it [00:02, 3507.22it/s]

9772it [00:02, 4062.15it/s]

10294it [00:02, 4351.67it/s]

10816it [00:03, 4095.07it/s]

11290it [00:03, 2568.69it/s]

11663it [00:03, 2751.77it/s]

12699it [00:03, 3448.74it/s]


In [24]:
# a row in 'data' contains:
# id of a document, id of a word in this document, a window around this word
# 1 + 1 + 10
data.shape[1]

12

In [25]:
# number of windows (equals to the total number of tokens)
data.shape[0]

1005435

# Get unigram distribution

In [26]:
word_counts = np.array(word_counts)
unigram_distribution = word_counts/sum(word_counts)

# Prepare word vectors

In [27]:
%%time
vocab_size = len(decoder)
embedding_dim = 50

# train a skip-gram word2vec model
texts = [[str(j) for j in doc] for i, doc in encoded_docs]
model = models.Word2Vec(texts, size=embedding_dim, window=5, workers=4, sg=1, negative=15, iter=70)
model.init_sims(replace=True)

word_vectors = np.zeros((vocab_size, embedding_dim)).astype('float32')
for i in decoder:
    word_vectors[i] = model.wv[str(i)]

c:\python37\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


KeyboardInterrupt: 

In [28]:
# number of unique words
vocab_size

NameError: name 'vocab_size' is not defined

# Prepare initialization for document weights

In [ ]:
texts = [[decoder[j] for j in doc] for i, doc in encoded_docs]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
%%time
n_topics = 25
lda = models.LdaModel(corpus, alpha=0.9, id2word=dictionary, num_topics=n_topics)
corpus_lda = lda[corpus]

In [ ]:
for i, topics in lda.show_topics(n_topics, formatted=False):
    print('topic', i, ':', ' '.join([t for t, _ in topics]))

In [ ]:
doc_weights_init = np.zeros((len(corpus_lda), n_topics))
for i in tqdm(range(len(corpus_lda))):
    topics = corpus_lda[i]
    for j, prob in topics:
        doc_weights_init[i, j] = prob

# Save data

In [20]:
np.save('data.npy', data)
np.save('word_vectors.npy', word_vectors)
np.save('unigram_distribution.npy', unigram_distribution)
np.save('decoder.npy', decoder)
np.save('doc_decoder.npy', doc_decoder)
np.save('doc_weights_init.npy', doc_weights_init)